# RLDT


## Step 1: Import the necessary libraries:


In [3]:
import numpy as np
import pandas as pd
import networkx as nx
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.nn.init as init
import itertools

In [41]:
num_IOT_devices = 10

voltages_frequencies_IOT = [
    (10e6  , 1.8),
    (20e6  , 2.3),
    (40e6  , 2.7),
    (80e6  , 4.0),
    (160e6 , 5.0),
]

num_MEC_devices = 5

voltages_frequencies_MEC = [
    (1500e6 ,  1.2),
    (1000e6 ,  1.0),
    (750e6, 0.825),
    (600e6, 0.8),
]

task_kinds = [1,2,3,4]

min_num_nodes_dag = 4
max_num_nodes_dag = 20
max_num_parents_dag = 5
num_dag_generations = 10000




## Step 2: Define the environment:


#### ALL THE DEVICES


In [42]:
import ast

devices = pd.read_csv("devices.csv")
devices["voltages_frequencies"] = devices["voltages_frequencies"].apply(lambda x: ast.literal_eval(x))
devices["capacitance"] = devices["capacitance"].apply(
    lambda x: ast.literal_eval(x)
)
devices["occupied_cores"] = devices["occupied_cores"].apply(
    lambda x: ast.literal_eval(x)
)
devices["powerIdle"] = devices["powerIdle"].apply(
    lambda x: ast.literal_eval(x)
)
devices["acceptableTasks"] = devices["acceptableTasks"].apply(
    lambda x: ast.literal_eval(x)
)
devices = devices.drop(["Unnamed: 0"],axis=1)
# devices

### Step 2.2: Application


#### _ALL THE TASKS_


In [43]:
def generate_random_dag(num_nodes):
    dag = nx.DiGraph()

    nodes = [f"t{i+1}" for i in range(num_nodes)]
    dag.add_nodes_from(nodes)

    available_parents = {node: list(nodes[:i]) for i, node in enumerate(nodes)}

    for i in range(2, num_nodes + 1):

        num_parents = min(
            random.randint(1, min(i, max_num_parents_dag)), len(
                available_parents[f"t{i}"])
        )

        # select parents
        parent_nodes = random.sample(available_parents[f"t{i}"], num_parents)
        # add parents
        dag.add_edges_from((parent_node, f"t{i}")
                           for parent_node in parent_nodes)

        # update available parents
        available_parents[f"t{i}"] = list(nodes[:i])

    return dag

In [44]:
def generate_task():
    tasks_data = []

    start_node_number = 1
    for run in range(num_dag_generations):

        num_nodes = random.randint(min_num_nodes_dag, max_num_nodes_dag)

        random_dag = generate_random_dag(num_nodes)

        mapping = {
            f"t{i}": f"t{i + start_node_number - 1}" for i in range(1, num_nodes + 1)
        }

        random_dag = nx.relabel_nodes(random_dag, mapping)
        for node in random_dag.nodes:
            parents = list(random_dag.predecessors(node))
            task_info = {
                "id": node,
                "job": run,
                "dependency": parents,
                "mobility": np.random.randint(1, 10),
                "kind": np.random.choice(task_kinds),
                "safe": np.random.choice([0, 1], p=[0.95, 0.05]),
                "computationalLoad": int(np.random.uniform(1, 11)*1e6),
                "dataEntrySize":int(np.random.uniform(1, 11)*1e6),
                "returnDataSize":int(np.random.uniform(1, 11)*1e6),
                "status": "READY",
            }
            tasks_data.append(task_info)
        start_node_number += num_nodes

    np.random.shuffle(tasks_data)
    tasks = pd.DataFrame(tasks_data)
    tasks = tasks.set_index("id")
    tasks_copy = tasks.copy()
    tasks_copy = tasks_copy.drop(["job","dependency","mobility","status"],axis=1)
    taskList = tasks_copy.index.tolist()
    return taskList, tasks_copy




## Step 4 : DDT


### Step 4.1: Initializing The tree


In [45]:
class DDT(nn.Module):
    def __init__(self, num_input, num_output, depth, max_depth):
        super(DDT, self).__init__()
        self.depth = depth
        self.max_depth = max_depth
        if depth != max_depth:
            # self.weights = nn.Parameter(torch.zeros(num_input))
            self.weights = nn.Parameter(torch.empty(
                num_input).normal_(mean=0, std=0.1))
            self.bias = nn.Parameter(torch.zeros(1))
        if depth == max_depth:
            self.prob_dist = nn.Parameter(torch.zeros(num_output))

        if depth < max_depth:
            self.left = DDT(num_input, num_output, depth + 1, max_depth)
            self.right = DDT(num_input, num_output, depth + 1, max_depth)

    def forward(self, x):
        if self.depth == self.max_depth:
            return self.prob_dist
        val = torch.sigmoid(torch.matmul(x, self.weights.t()) + self.bias)
        a = np.random.uniform(0, 1)
        if a < 0.1:
            val = 1 - val
        if val >= 0.5:

            return val * self.right(x)
        else:

            return (1 - val) * self.left(x)

## Step 5: RL


In [46]:
def calc_execution_time(device, task, core, dvfs):
    if device['id'] == "cloud":
        return task["computationalLoad"] / device["voltages_frequencies"][0]
    else:
        return task["computationalLoad"] / device["voltages_frequencies"][core][dvfs][0]


def calc_power_consumption(device, task, core, dvfs):
    if device['id'] == "cloud":
        return 13.85 * calc_execution_time(device, task, core, dvfs)
    return (
        device["capacitance"][core]
        * (device["voltages_frequencies"][core][dvfs][1] ** 2)
        * device["voltages_frequencies"][core][dvfs][0]
    )
def calc_energy(device, task, core, dvfs):
    return calc_execution_time(device, task, core, dvfs) * calc_power_consumption(device, task, core, dvfs)


def calc_total(device, task, core, dvfs):
    timeTransMec = 0
    timeTransCC = 0
    exeTime = 0
    e = 0

    transferRate5g =1e9
    latency5g=5e-3
    transferRateFiber =1e10
    latencyFiber=1e-3

    timeDownMec = task["returnDataSize"] / transferRate5g
    timeDownMec += latency5g
    timeUpMec = task["dataEntrySize"] / transferRate5g
    timeUpMec += latency5g

    alpha = 52e-5
    beta = 3.86412
    powerMec = alpha * 1e9 / 1e6 + beta

    timeDownCC = task["returnDataSize"] / transferRateFiber
    timeDownCC += latencyFiber
    timeUpCC = task["dataEntrySize"] / transferRateFiber
    timeUpCC += latencyFiber

    powerCC = 3.65 


    if device["id"].startswith("mec"):
        timeTransMec =  timeUpMec +  timeDownMec 
        energyTransMec = powerMec *  timeTransMec
        exeTime = calc_execution_time(device, task, core, dvfs)
        totalTime = exeTime + timeTransMec 
        e = calc_energy(device, task, core, dvfs)
        totalEnergy =  e + energyTransMec

    elif device['id'].startswith("cloud"):
        timeTransMec =  timeUpMec +  timeDownMec 
        energyTransMec = powerMec * timeTransMec
        
        timeTransCC = timeUpCC+timeDownCC
        energyTransCC =  powerCC * timeTransCC
        
        exeTime = calc_execution_time(device, task, core, dvfs)
        totalTime =  exeTime + timeTransMec +timeTransCC

        e = calc_energy(device, task, core, dvfs)
        totalEnergy =  + energyTransMec + energyTransCC

    elif device['id'].startswith("iot"):
        exeTime = calc_execution_time(device, task, core, dvfs)
        totalTime = exeTime
        e = calc_energy(device, task, core, dvfs)
        totalEnergy = e

    return totalTime , totalEnergy

In [47]:
def getPunish(rSetup):
    match rSetup:
        case "00":
            p = 10
        case "01":
            p = 10
        case "02":
            p = 100
        case "03":
            p = 100
        case "04":
            p = 20
        case "05":
            p = 20
        case "06":
            p = 15
        case "07":
            p = 15
        case "08":
            p = 10
        case "09":
            p = 10
    
    return p

In [48]:
def checkIfSuitable(state, device, punish):
    punishment = 0
    safeFail = 0
    taskFail = 0
    if  state['safe'] and not device["handleSafeTask"]:
        punishment += 1
        safeFail += 1
        
    if state['kind'] not in device["acceptableTasks"]:
        punishment += 1
        taskFail += 1
    # return taskFail, safeFail
    return (punishment if punishment > 0 else 0, taskFail, safeFail)

In [49]:
def getSetup(t, e, setup, alpha=1, beta=1):
    match setup:
        case "00":
            reward = -1 * (e + t)
        case "01":
            reward = -1 * (alpha * e + beta * t)
        case "02":
            reward = -1 / (e + t)
        case "03":
            reward = -1 / (alpha * e + beta * t)
        case "04":
            reward = -np.exp(e) - np.exp(t)
        case "05":
            reward = -np.exp(alpha * e) - np.exp(beta * t)
        case "06":
            reward = -np.exp(t + e)
        case "07":
            reward = -np.exp(alpha * t + beta * e)
        case "08":
            reward = np.exp(-t - e)
        case "09":
            reward = np.exp(-1 * (alpha * t + beta * e))
    
    return reward

In [50]:
class Environment:
    def __init__(self):
        self.rSetup = "01"
        self.punish = 0
        self.alpha = 1
        self.beta = 1
        self.last_epoch_t = 0
        self.last_epoch_l = 0
        self.last_epoch_e = 0
        self.taskList = []
        self.tasks_copy = None
        self.totalSafeFail = 0
        self.totalTaskFail = 0
        self.totalReward = 0
        self.feature_size = 5
        self.num_actions = len(devices)
        self.max_depth = 3
        self.agent = DDT(self.feature_size, self.num_actions,
                         depth=0, max_depth=self.max_depth)
        self.optimizer = optim.Adam(self.agent.parameters(), lr=0.005)
        self.avg_time_history = []
        self.avg_energy_history = []
        self.avg_fail_history = []
        self.avg_safe_fail_history = []
        self.avg_task_fail_history = []
     
    def execute_action(self, state, action):
        self.taskList.pop(0)
        device = devices.iloc[action]        

        punishment, taskFail, safeFail = checkIfSuitable(state, device, self.punish)
        # taskFail, safeFail = checkIfSuitable(state, device)
        self.punish *= punishment

        if safeFail:
            self.totalSafeFail += 1
        if taskFail:
            self.totalTaskFail += 1


        if not (self.punish):
            for coreIndex in range(len(device["occupied_cores"])):
                if device["occupied_cores"][coreIndex] == 0:
                    total_t, total_e  = calc_total(device, state, coreIndex,np.random.randint(0,3))
                    reward = getSetup(total_t, total_e, self.rSetup, alpha=self.alpha, beta=self.beta)
                    # reward = -1 / (total_t + total_e)
                    # print(f"device {device['id']} ////// time {total_t}  ////// energy {total_e} ")
                    self.totalReward += reward
                    return (self.tasks_copy.loc[self.taskList[0]], reward, total_t, total_e)
        self.totalFail += 1
        return (self.tasks_copy.loc[self.taskList[0]], self.punish, 0, 0)


    def train(self, num_epoch, num_episodes):

        total_avg_t = 0
        total_avg_e = 0
        total_avg_r = 0
        total_avg_l = 0
        
        self.safeFailHistory = []
        self.taskFailHistory = []
        self.lossHistory = []
        
        for i in range(num_epoch):
            


            total_loss = 0
            self.totalFail = 0
            self.totalTaskFail = 0
            self.totalSafeFail = 0
            self.totalReward = 0
            total_loss = 0
            total_reward = 0
            totalTime = 0
            totalEnergy = 0


            
            for j in range(num_episodes):
                state = self.tasks_copy.loc[self.taskList[0]]
                x = torch.tensor(np.array(state.values, dtype=np.float32)).unsqueeze(0)
                
                output = self.agent(x)
                action_probabilities = torch.softmax(output, dim=0)
                action_index = torch.multinomial(action_probabilities, 1).item()
                # action_index = torch.argmax(action_probabilities).item()

                next_state, reward, t, e = self.execute_action(state, action_index)
                loss = (output[action_index] * reward)

                total_reward += reward
                total_loss += loss
                
                totalTime += t
                totalEnergy += e
                
                
            
            self.optimizer.zero_grad()
            avg_loss = total_loss/num_episodes
            self.lossHistory.append(avg_loss)
            avg_time = totalTime / num_episodes
            avg_energy = totalEnergy / num_episodes
            
            # avg_reward = total_reward / num_episodes
            avg_reward = self.totalReward / num_episodes
            avg_loss = total_loss/num_episodes

            self.avg_time_history.append(avg_time)
            self.avg_energy_history.append(avg_energy)
            self.avg_fail_history.append(self.totalFail)
            self.avg_safe_fail_history.append(self.totalSafeFail)
            self.avg_task_fail_history.append(self.totalTaskFail)


            total_avg_t += avg_time
            total_avg_e += avg_energy
            total_avg_l += avg_loss
            total_avg_r += avg_reward
            

            avg_loss.backward()
            self.optimizer.step()
            if i % 1 == 0:
                # print(f"Epoch {i+1} // avg cc time: {avg_cc} // avg mec: {avg_mec} // avg og time: {avg_og} total fail: {env.totalFail} // Average Loss: {avg_loss}// ")
                # print(f"Epoch {i+1} // avg time: {avg_time} // avg added Time: {avg_added_time} // avg og time: {avg_og} total fail: {env.totalFail} // Average Loss: {avg_loss}// ")
                # print(f"Epoch {i+1}  // safe/task fail: {self.totalSafeFail}/{self.totalTaskFail} // Average Loss: {avg_loss:.2f} // Total Reward: {self.totalReward:.2f} // Average Reward: {avg_reward:.2f} // Avg time: {avg_time:.2f} // Avg energy: {avg_energy:.2f}")
                pass
            
            if i == 10000:

                self.last_epoch_t = avg_time
                self.last_epoch_l = avg_loss
                self.last_epoch_e = avg_energy
                
            
                
            #     print(f"safe/task fail: {env.totalSafeFail}/{env.totalTaskFail} // Average Loss: {avg_loss:.2f} // Total Reward: {env.totalReward:.2f} // Average Reward: {avg_reward:.2f} // Avg time: {avg_time:.2f} // Avg energy: {avg_energy:.2f}")

                # env.totalFail = 0
            self.taskFailHistory.append(self.totalTaskFail)
            self.safeFailHistory.append(self.totalSafeFail)
            self.totalSafeFail = 0
            self.totalTaskFail = 0

        

        

        avg_avg_t = total_avg_t / num_epoch
        avg_avg_l = total_avg_l / num_epoch
        avg_avg_r = total_avg_r / num_epoch
        avg_avg_e = total_avg_e / num_epoch

        

        safe_fail_indices = [i for i, x in enumerate(self.safeFailHistory) if x != 0]
        if safe_fail_indices:
            last_safe_index = safe_fail_indices[-1] + 1
        
        task_fail_indices = [i for i, x in enumerate(self.taskFailHistory) if x != 0]
        if task_fail_indices:
            last_task_index = task_fail_indices[-1] + 1
        
        task_fail_percent = len(task_fail_indices) / num_epoch
        safe_fail_percent = len(safe_fail_indices) / num_epoch
        is_loss_min = 1 if self.last_epoch_l < min(self.lossHistory) else 0

        first_10_avg_time = np.mean(self.avg_time_history[:10])
        first_10_avg_energy = np.mean(self.avg_energy_history[:10])
        first_10_avg_fail = np.mean(self.avg_fail_history[:10])
        first_10_avg_safe_fail = np.mean(self.avg_safe_fail_history[:10])
        first_10_avg_task_fail = np.mean(self.avg_task_fail_history[:10])

        last_10_avg_time = np.mean(self.avg_time_history[-10:])
        last_10_avg_energy = np.mean(self.avg_energy_history[-10:])
        last_10_avg_fail = np.mean(self.avg_fail_history[-10:])
        last_10_avg_safe_fail = np.mean(self.avg_safe_fail_history[-10:])
        last_10_avg_task_fail = np.mean(self.avg_task_fail_history[-10:])

        
        new_epoch_data = {
            "Setup": [self.rSetup],
            "Punishment": [self.punish],
            "Alpha": [self.alpha],
            "Beta": [self.beta],
            "Average Loss": [avg_avg_l.detach().numpy() if isinstance(avg_avg_l, torch.Tensor) else avg_avg_l],
            "Last Epoch Loss": [self.last_epoch_l.detach().numpy() if isinstance(self.last_epoch_l, torch.Tensor) else self.last_epoch_l],
            "is Loss min": [is_loss_min.detach().numpy() if isinstance(is_loss_min, torch.Tensor) else is_loss_min],
            "Task Converge": [last_task_index.detach().numpy() if isinstance(last_task_index, torch.Tensor) else last_task_index],
            "Task Fail Percentage": [task_fail_percent.detach().numpy() if isinstance(task_fail_percent, torch.Tensor) else task_fail_percent],
            "Safe Converge": [last_safe_index.detach().numpy() if isinstance(last_safe_index, torch.Tensor) else last_safe_index],
            "Safe Fail Percentage": [safe_fail_percent.detach().numpy() if isinstance(safe_fail_percent, torch.Tensor) else safe_fail_percent],
            "Average Time": [avg_avg_t.detach().numpy() if isinstance(avg_avg_t, torch.Tensor) else avg_avg_t],
            "Last Epoch Time": [self.last_epoch_t.detach().numpy() if isinstance(self.last_epoch_t, torch.Tensor) else self.last_epoch_t],
            "Average Energy": [avg_avg_e.detach().numpy() if isinstance(avg_avg_e, torch.Tensor) else avg_avg_e],
            "Last Epoch Energy": [self.last_epoch_e.detach().numpy() if isinstance(self.last_epoch_e, torch.Tensor) else self.last_epoch_e],
            "Average Reward": [avg_avg_r.detach().numpy() if isinstance(avg_avg_r, torch.Tensor) else avg_avg_r],
            "Total Reward": [total_reward.detach().numpy() if isinstance(total_reward, torch.Tensor) else total_reward],

            "First 10 Avg Time": [first_10_avg_time.detach().numpy() if isinstance(first_10_avg_time, torch.Tensor) else first_10_avg_time],
            "Last 10 Avg Time": [last_10_avg_time.detach().numpy() if isinstance(last_10_avg_time, torch.Tensor) else last_10_avg_time],
            "First 10 Avg Energy": [first_10_avg_energy.detach().numpy() if isinstance(first_10_avg_energy, torch.Tensor) else first_10_avg_energy],
            "Last 10 Avg Energy": [last_10_avg_energy.detach().numpy() if isinstance(last_10_avg_energy, torch.Tensor) else last_10_avg_energy],
            "First 10 Avg Safe Fail": [first_10_avg_safe_fail.detach().numpy() if isinstance(first_10_avg_safe_fail, torch.Tensor) else first_10_avg_safe_fail],
            "Last 10 Avg Safe Fail": [last_10_avg_safe_fail.detach().numpy() if isinstance(last_10_avg_safe_fail, torch.Tensor) else last_10_avg_safe_fail],
            "First 10 Avg Task Fail": [first_10_avg_task_fail.detach().numpy() if isinstance(first_10_avg_task_fail, torch.Tensor) else first_10_avg_task_fail],
            "Last 10 Avg Task Fail": [last_10_avg_task_fail.detach().numpy() if isinstance(last_10_avg_task_fail, torch.Tensor) else last_10_avg_task_fail]
        }   

        df = pd.read_csv("data2.csv")

        # Convert the new data into a DataFrame and concatenate it
        new_df = pd.DataFrame(new_epoch_data)
        df = pd.concat([df, new_df], ignore_index=True)

        # Save the updated DataFrame back to CSV
        df.to_csv("data2.csv", index=False)


        # print(f'Overall Average Time across all epochs: {avg_avg_t}')
        # print(f'Overall Average e across all epochs: {avg_avg_e:.2f}')
        # print(f'Overall Average l across all epochs: {avg_avg_l:.2f}')
        # print(f'Overall Average r across all epochs: {avg_avg_r:.2f}')
     





# env.totalAddedAvg += avg_cc


# env = Environment()
# tree = env.agent
# env.train(1001, 10)

# print('///////////////////')

# for name, param in env.agent.named_parameters():
#     if "prob_dist" or "bias" not in name:
#         # print(name,param)
#         pass

In [51]:

def train_test():
    rpSetup_list = {"01": [1, 10, 100, 1000], "03": [10, 100, 1000, 10000], "05": [2, 20, 200, 2000], "07": [1.5, 15, 150, 1500], "09": [1, 10, 100, 1000]}
    alpha_list = [1, 2, 5, 10, 20]
    beta_list = [1, 2, 5, 10, 20]
    
    for reward, punishments in rpSetup_list.items():
        for punish, alpha, beta in itertools.product(punishments, alpha_list, beta_list):   
            if alpha == beta and alpha != 1:
                continue
            
            taskList, tasks_copy = generate_task()
            env = Environment()
            env.rSetup = reward
            env.alpha = alpha
            env.beta = beta
            env.punish = punish
            env.taskList = taskList
            env.tasks_copy = tasks_copy
            
            tree = env.agent
            env.train(10001, 10)

train_test()   
print("completed")


/var/folders/_r/7fw_p5qd057b2drkmtmpn2l80000gn/T/ipykernel_68515/2886625192.py:217: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, new_df], ignore_index=True)


In [ ]:
# import pandas as pd

# # Define the column headers as a list
# headers = [
#     "Setup", "Punishment", "Alpha", "Beta","Average Loss", "Last Epoch Loss", "is Loss min", "Task Converge", "Task Fail Percentage", "Safe Converge", "Safe Fail Percentage", "Average Time","Last Epoch Time",
#     "Average Energy", "Last Epoch Energy", "Average Reward", "Total Reward", "First 10 Avg Time", "Last 10 Avg Time","First 10 Avg Energy", "Last 10 Avg Energy","First 10 Avg Safe Fail", "Last 10 Avg Safe Fail","First 10 Avg Task Fail", "Last 10 Avg Task Fail"
# ]
# # Create an empty DataFrame with these headers
# df = pd.DataFrame(columns=headers)

# # Specify the filename
# filename = "data2.csv"

# # Save the DataFrame to a CSV file
# df.to_csv(filename, index=False)

# print(f"CSV file '{filename}' created successfully with headers only.")

CSV file 'data2.csv' created successfully with headers only.


In [ ]:


# Load the data
df = pd.read_csv("data.csv")

# Sort the DataFrame by 'setup' column
df = df.sort_values(by='Setup')

# Save the sorted DataFrame back to the CSV
df.to_csv("data.csv", index=False)

print("CSV file sorted by 'setup'.")
